In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize images
])

# Load dataset using ImageFolder (auto-labels folders as class labels)
def load_dataset(data_path, batch_size=16, shuffle=True):
    dataset = datasets.ImageFolder(root=data_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader, dataset.classes  # Also return class names

# Load Train, Test, Validation datasets
train_loader, class_names = load_dataset("Train", batch_size=16)
test_loader, _ = load_dataset("Test", batch_size=16, shuffle=False)
val_loader, _ = load_dataset("Validation", batch_size=16, shuffle=False)

# Print class names (should be ['normal', 'stroke'])
print("Classes:", class_names)

Classes: ['Normal', 'Stroke']


In [ ]:
import torch
import torch.nn as nn

def create_srgan_generator():
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
        nn.Tanh()
    )
    return model

def train_srgan(epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_srgan_generator().to(device)  
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

    for epoch in range(epochs):
        print(f"Training Epoch {epoch+1}/{epochs}...")
    
    torch.save(model.state_dict(), "srgan_model.pth")
    print("✅ SRGAN Model Saved!")

train_srgan()


In [7]:
def create_denoising_generator():
    return nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(64, 3, kernel_size=3, padding=1)
    )

def train_denoising_gan(epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_denoising_generator().to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

    for epoch in range(epochs):
        for imgs, _ in train_loader:
            noisy_imgs = imgs + 0.1 * torch.randn_like(imgs)  # Add noise
            noisy_imgs, imgs = noisy_imgs.to(device), imgs.to(device)

            output = model(noisy_imgs)
            loss = criterion(output, imgs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}: Loss {loss.item()}")

    torch.save(model, "denoising_model.pth")
    print("✅ Denoising Model Saved!")

train_denoising_gan()



Epoch 1: Loss 0.021207906305789948
Epoch 2: Loss 0.013007164932787418
Epoch 3: Loss 0.006572111509740353
Epoch 4: Loss 0.004941129591315985
Epoch 5: Loss 0.004091570619493723
Epoch 6: Loss 0.0032092754263430834
Epoch 7: Loss 0.0027026108000427485
Epoch 8: Loss 0.0022438964806497097
Epoch 9: Loss 0.0025238629896193743
Epoch 10: Loss 0.0019385587656870484
Epoch 11: Loss 0.001775426440872252
Epoch 12: Loss 0.0018056790577247739
Epoch 13: Loss 0.0015430934727191925
Epoch 14: Loss 0.0018672632286325097
Epoch 15: Loss 0.001385255018249154
Epoch 16: Loss 0.0015486415941268206
Epoch 17: Loss 0.0017663551261648536
Epoch 18: Loss 0.0016322728479281068
Epoch 19: Loss 0.00148071872536093
Epoch 20: Loss 0.0013375234557315707
Epoch 21: Loss 0.0013909657718613744
Epoch 22: Loss 0.0009587527019903064
Epoch 23: Loss 0.00128195493016392
Epoch 24: Loss 0.001232074573636055
Epoch 25: Loss 0.001221893006004393
Epoch 26: Loss 0.0012226564576849341
Epoch 27: Loss 0.0013223313726484776
Epoch 28: Loss 0.001103

In [8]:
def create_cyclegan_generator():
    return nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(64, 3, kernel_size=3, padding=1)
    )

def train_cyclegan(epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_cyclegan_generator().to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

    for epoch in range(epochs):
        for imgs, _ in train_loader:
            imgs = imgs.to(device)
            output = model(imgs)
            loss = criterion(output, imgs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}: Loss {loss.item()}")

    torch.save(model, "cyclegan_model.pth")
    print("✅ CycleGAN Model Saved!")

train_cyclegan()


Epoch 1: Loss 0.017321070656180382
Epoch 2: Loss 0.009665808640420437
Epoch 3: Loss 0.005323986988514662
Epoch 4: Loss 0.0038219131529331207
Epoch 5: Loss 0.0025817251298576593
Epoch 6: Loss 0.0020207948982715607
Epoch 7: Loss 0.00126533186994493
Epoch 8: Loss 0.0010422603227198124
Epoch 9: Loss 0.0007417536107823253
Epoch 10: Loss 0.0006573192658834159
Epoch 11: Loss 0.0006549193640239537
Epoch 12: Loss 0.00038285410846583545
Epoch 13: Loss 0.00025636624195612967
Epoch 14: Loss 0.00029580690898001194
Epoch 15: Loss 0.0002305494708707556
Epoch 16: Loss 0.0003672844322863966
Epoch 17: Loss 0.00017250275413971394
Epoch 18: Loss 0.00022744493617210537
Epoch 19: Loss 0.00029675077530555427
Epoch 20: Loss 0.00023643599706701934
Epoch 21: Loss 0.00024407453020103276
Epoch 22: Loss 0.0001901862706290558
Epoch 23: Loss 0.0002247558004455641
Epoch 24: Loss 0.00017694482812657952
Epoch 25: Loss 0.00018207848188467324
Epoch 26: Loss 0.00012264672841411084
Epoch 27: Loss 0.00010450708941789344
Epo

In [ ]:
@app.post("/predict/srgan/")
async def predict(file: UploadFile = File(...)):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_srgan_generator().to(device)
    
    # Load the model with map_location to handle CPU-only machines
    model.load_state_dict(torch.load("srgan_model.pth", map_location=device))
    model.eval()

    # Read and preprocess the input image
    image = Image.open(file.file).convert('RGB')
    transform = transforms.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Perform inference
    with torch.no_grad():
        output = model(image_tensor)

    # Convert output tensor to image
    output_image = transforms.ToPILImage()(output.squeeze(0).cpu())

    # Save or return the output image
    output_image.save("output_image.png")
    
    return {"message": "Super-resolution image generated and saved as output_image.png"}